In [1]:
import FINE as fn
from getData import getData
import pandas as pd
import os
import copy
cwd = os.getcwd()
data = getData()


%matplotlib inline  
%load_ext autoreload
%autoreload 2

In [2]:
locations = {'EnergyLand_elec'}
commodityUnitDict = {'electricity': r'GW$_{el}$'}
commodities = {'electricity'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

In [3]:
esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                           commodityUnitsDict=commodityUnitDict,
                           hoursPerTimeStep=1, costUnit='1e6 Euro', lengthUnit='km', verboseLogLevel=0)

# Sources

In [4]:
esM.add(fn.Source(esM=esM, name='Wind_Onshore', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind_onshore, operationRateMax'],
                  capacityMax=data['Wind_onshore, capacityMax'],
                  investPerCapacity=1250, opexPerCapacity=1250*0.02, interestRate=0.08,
                  economicLifetime=20, technicalLifetime=10))

esM.add(fn.Source(esM=esM, name='Wind_Onshore_stock', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind_onshore, operationRateMax'],
                  capacityFix=100, 
                  investPerCapacity=1250, opexPerCapacity=1250*0.02, interestRate=0.08,
                  economicLifetime=20))

esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'],
                  capacityMax=data['PV, capacityMax'],
                  investPerCapacity=800, opexPerCapacity=800*0.019, interestRate=0.08,
                  economicLifetime=20))

In [5]:
esM.add(fn.Source(esM=esM, name='el_Import', commodity='electricity', hasCapacityVariable=False,
                  operationRateMax=data['el_Import, operationRateMax']))

# Storage

In [6]:
esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.99,
                   dischargeEfficiency=0.99, selfDischarge=0.004,
                   doPreciseTsaModeling=False,investPerCapacity=120, 
                   opexPerCapacity=120*0.014, opexPerChargeOperation=0.0001, 
                   interestRate=0.08, economicLifetime=10))

esM.add(fn.Storage(esM=esM, name='Li-ion batteries_stock', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.99,
                   dischargeEfficiency=0.99, selfDischarge=0.004,
                   doPreciseTsaModeling=False,investPerCapacity=120, 
                   opexPerCapacity=120*0.014, opexPerChargeOperation=0.0001, 
                   interestRate=0.08, economicLifetime=10, capacityFix=0))

# Sink

In [7]:
eDemand=516
esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']*eDemand))

# Optimization

In [8]:
esM.componentNames

{'Wind_Onshore': 'SourceSinkModel',
 'el_Import': 'SourceSinkModel',
 'Li-ion batteries': 'StorageModel',
 'Electricity demand': 'SourceSinkModel'}

In [9]:
esM.optimizeMyopic(startYear=2020, nbOfSteps=3, stepLength=5, numberOfTypicalPeriods=7, timeSeriesAggregation=True)


Clustering time series data with 7 typical periods and 24 time steps per period...
		(0.4842 sec)

Time series aggregation specifications:
Number of typical periods:7, number of time steps per periods:24

Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.6405 sec)

Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0625 sec)

Declaring shared potential constraint...
		(0.0000 sec)

Declaring commodity balances...
		(0.0156 sec)

Declaring objective function...
		(0.0000 sec)

Using license file C:\Users\there\gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file C:\Users\there\AppData\Local\Temp\tmpgdh3eqcr.pyomo.lp
Reading time = 0.02 seconds
x1400: 2618 rows, 1400 columns, 6462 nonzeros
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value

In [ ]:
esM.componentModelingDict['SourceSinkModel'].componentsDict['Wind_Onshore_stock'].capacityFix

In [ ]:
esM.cluster(numberOfTypicalPeriods=7)

In [ ]:
esM.optimize(timeSeriesAggregation=True, solver='gurobi')

In [ ]:
esM.getOptimizationSummary("SourceSinkModel", outputLevel=0)

In [ ]:
esM.getStock(2025)

In [ ]:
esM.componentModelingDict['SourceSinkModel'].componentsDict['Wind_Onshore'].name

In [ ]:
esM.getOptimizationSummary("StorageModel", outputLevel=2)

# Jetzt kommt der Stock!!

In [ ]:
esM.getStock('2020')

In [ ]:
esM.componentModelingDict['SourceSinkModel'].componentsDict['Wind_Onshore_stock'].capacityFix

In [ ]:
esM.declareOptimizationProblem(timeSeriesAggregation=False)

# TESTEN

In [ ]:
esM.setFixedVariables('capacityVariablesOptimum')

In [ ]:
compValues = esM.componentModelingDict['StorageModel'].getOptimizedValues('capacityVariablesOptimum')

In [ ]:
compValues

In [ ]:
stockComp = copy.copy(esM.componentModelingDict['StorageModel'].componentsDict['Li-ion batteries'])

In [ ]:
stockComp

In [ ]:
esM.componentModelingDict['StorageModel'].componentsDict['Li-ion batteries']

In [ ]:
stockComp.name = stockComp.name + '_stock'

In [ ]:
esM.componentModelingDict['StorageModel'].componentsDict['Li-ion batteries'].name

In [ ]:
stockComp.name

In [ ]:
compValues.loc['Wind_Onshore'].values

In [ ]:
compValues.loc['Li-ion batteries'][0]

In [ ]:
for mdl in esM.componentModelingDict.keys():
    print(mdl)

In [ ]:
for comp in esM.componentModelingDict[mdl].componentNames.keys():
        print(comp)

In [ ]:
for comp in esM.componentModelingDict[mdl].componentsDict.keys():
    print(comp)

In [ ]:
esM.getStock()

In [ ]:
esM.componentModelingDict[mdl].componentsDict['Li-ion batteries'].name

In [ ]:
stockComp = esM.componentModelingDict[mdl].componentsDict['Li-ion batteries']

In [ ]:
stockComp.dischargeEfficiency

In [ ]:
stockComp2.name = stockComp.name+'_Hallo'

In [ ]:
stockComp2.operationRateMax

In [ ]:
stockComp.capacityFix = compValues.loc['Li-ion batteries'][0]

In [ ]:
stockComp.capacityFix

In [ ]:
esM.add(stockComp)

In [ ]:
for comp in esM.componentModelingDict['SourceSinkModel'].componentsDict.keys():
    print(comp)

In [ ]:
stockComp2 = esM.componentModelingDict['SourceSinkModel'].componentsDict['Wind_Onshore']

In [ ]:
stockComp2

In [ ]:
stockComp2.capacityMax = None

In [ ]:
esM.componentModelingDict['SourceSinkModel'].getOptimizedValues('capacityVariablesOptimum').loc['Wind_Onshore']

In [ ]:
compValues.values[0]

In [ ]:
stockComp2.capacityFix = compValues.values[0]

In [ ]:
stockComp2.name = stockComp2.name+'_stock'

In [ ]:
esM.add(stockComp2)

In [ ]:
stockComp4 = stockComp2

In [ ]:
stockComp4.name

In [ ]:
stockComp2.name

In [ ]:
stockComp4.name = 'Tschuess'

In [ ]:
stockComp2.name

In [ ]:
stockComp3 = copy.copy(stockComp2)

In [ ]:
stockComp3.name

In [ ]:
stockComp2.name

In [ ]:
stockComp3.name = 'Hallo'

In [ ]:
stockComp2.name